In [1]:
import pandas as pd
with open('haystack.txt', 'r', encoding='utf-8') as file:
    text = file.read()

In [2]:
list_text = text.split('\n')

In [3]:
len(list_text)

599300

In [21]:
from langchain_text_splitters import CharacterTextSplitter, RecursiveCharacterTextSplitter

text_splitter = CharacterTextSplitter(
    separator="\n\n",
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
)

rec_text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=1000,
    chunk_overlap=200,
    length_function=len,
    is_separator_regex=False,
    separators=["\n\n", "\n", "\n\n\n", "."],
)

In [31]:
texts_rec = rec_text_splitter.create_documents([text])

In [58]:
print(texts_rec[2256])

page_content='hooded head moved only when he cocked an ear at an 
interesting sound. 

Through the day’s gathering crowds the strange pair 
came, arriving at last on the steps which led up like terraced 
hectares to the escarpment which was Alia’s Temple, a 
fitting companion to Paul’s Keep. Up the steps The Preacher 
went until he and his young guide came to the third landing, 
where pilgrims of the Hajj awaited the morning opening of 
those gigantic doors above them. They were doors large 
enough to have admitted an entire cathedral from one of 
the ancient religions. Passing through them was said to 
reduce a pilgrim’s soul to motedom, sufficiently small that it 
could pass through the eye of a needle and enter heaven.'


In [61]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-large",
    # With the `text-embedding-3` class
    # of models, you can specify the size
    # of the embeddings you want returned.
    # dimensions=1024
)

RuntimeError: no validator found for <class 'langchain_core.callbacks.base.BaseCallbackHandler'>, see `arbitrary_types_allowed` in Config

In [28]:
texts = text_splitter.create_documents([text])

Created a chunk of size 1438, which is longer than the specified 1000
Created a chunk of size 1048, which is longer than the specified 1000
Created a chunk of size 1018, which is longer than the specified 1000
Created a chunk of size 1058, which is longer than the specified 1000
Created a chunk of size 1264, which is longer than the specified 1000
Created a chunk of size 1052, which is longer than the specified 1000
Created a chunk of size 1180, which is longer than the specified 1000
Created a chunk of size 1038, which is longer than the specified 1000
Created a chunk of size 1056, which is longer than the specified 1000
Created a chunk of size 1167, which is longer than the specified 1000
Created a chunk of size 1325, which is longer than the specified 1000
Created a chunk of size 1028, which is longer than the specified 1000
Created a chunk of size 1449, which is longer than the specified 1000
Created a chunk of size 1029, which is longer than the specified 1000
Created a chunk of s

In [29]:
print(texts[100])

page_content='Paul lifted the companion rapier, bent it in his 
hands, stood in the agui/e, one foot forward. He let 
his manner go solemn in a comic imitation of Dr. 
Yueh. 

“What a dolt my father sends me for weaponry,” 
Paul intoned. “This doltish Gurney Halleck has 
forgotten the first lesson for a fighting man armed 
and shielded.” Paul snapped the force button at his 
waist, felt the crinkled-skin tingling of the defensive 
field at his forehead and down his back, heard 
external sounds take on characteristic shield-filtered 
flatness. “In shield fighting, one moves fast on 


defense, slow on attack,” Paul said. “Attack has the 
sole purpose of tricking the opponent into a misstep, 
setting him up for the attack sinister. The shield turns 
the fast blow, admits the slow kindjal!” Paul snapped 
up the rapier, feinted fast and whipped it back for a 
Slow thrust timed to enter a shield’s mindless 
defenses.'
